# Testa ut en modell för att transformera streck till odds.  
Används för min specialvariant av Kelly 

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
from IPython.display import display
pd.set_option('display.width', 100)

In [3]:
df = pd.read_csv('..\\all_data.csv')
# Följande datum saknar avd==5 och kan inte användas
saknas = ['2015-08-15', '2016-08-13', '2017-08-12']
df = df[~df.datum.isin(saknas)]
X = df.copy()
X.drop('plac', axis=1, inplace=True)
# X = ordinal_enc(X, 'häst')
y = (df.plac == 1)*1   # plac 1 eller 0

In [4]:
def model_streck_to_odds(X_):
    X = X_.copy()
    # import modules for linear regression
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_absolute_error as mae
    # import random forest module
    from sklearn.ensemble import RandomForestRegressor

    X_odds = X.loc[X.vodds <= 40]  # remove outliers
    ix_break = int(len(X_odds.datum.unique())*0.75)
    test_start = X_odds.datum.unique()[ix_break]

    X_train, X_test = X_odds[X_odds.datum <
                             test_start], X_odds[X_odds.datum >= test_start]
    y_train, y_test = X_train['vodds'], X_test['vodds']
    X_train = X_train[['streck']].astype(float)
    X_test = X_test[['streck']].astype(float)

    # make a model of RF
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, random_state=0)
    rf.fit(X_train, y_train)
    y_predrf = rf.predict(X_test)
    # make a model and fit it
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_predlr = linreg.predict(X_test)

    # print the coefficients
    print('Coefficients:', linreg.coef_)
    # print the mean absolute error
    print("LR Mean absolute error: %.2f" % mae(y_test, y_predlr))
    print("RF Mean absolute error: %.2f" % mae(y_test, y_predrf))

    return linreg, rf


linreg, rf = model_streck_to_odds(X)   # used in next cell

Coefficients: [-0.52621488]
LR Mean absolute error: 6.08
RF Mean absolute error: 2.60


In [5]:
def kelly(proba, streck, odds):  # proba = prob winning, streck i % = streck
    if odds is None:
        o = rf.predict(streck.copy())
    else:
        o = rf.predict(streck.copy())

    # for each values > 50 in odds set to 1
    o[o > 40] = 1
    return (o*proba - (1-proba))/o


In [8]:
# test kelly
df['oddsl'] = linreg.predict(df[['streck']])
df['oddsr'] = rf.predict(df[['streck']])
df.loc[df.vodds<10][['streck','vodds','oddsl','oddsr']]

,streck,vodds,oddsl,oddsr
1,7.0,8.21,16.318115,11.331575
2,23.0,2.92,7.898677,3.742706
3,48.0,2.35,-5.256696,2.046252
13,11.0,8.11,14.213255,7.894903
19,67.0,1.70,-15.254778,1.512548
...,...,...,...,...
43796,12.0,6.97,13.687040,7.584810
43799,29.0,2.99,4.741387,3.150283
43801,17.0,5.69,11.055966,5.143853
43803,19.0,5.27,10.003536,4.631335
